In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from keras import backend as K

In [2]:
data = pd.read_csv("./pison_data_interview.csv", header=None, names=["timestamp","chan0_raw","chan1_raw","chan0_hp","chan1_hp","quat_x","quat_y","quat_z","quat_w","gyro_x","gyro_y","gyro_z","acc_x","acc_y","acc_z","label","rep"])


In [3]:
data = data[["chan0_hp","chan1_hp","gyro_x","gyro_y","gyro_z","acc_x","acc_y","acc_z"]]


In [4]:
data.shape

(14981, 8)

In [5]:
def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [6]:
# define input timeseries
timeseries = np.array([data.chan0_hp, data.chan1_hp, data.gyro_x, data.gyro_y, data.gyro_z, data.acc_x, data.acc_y, data.acc_z]).transpose()

timesteps = timeseries.shape[0]
n_features = timeseries.shape[1]

timeseries

array([[-1.1889000e+04,  1.7295000e+04, -1.3428699e+00, ...,
        -6.6015625e-01, -1.0003906e+01,  1.2187500e+00],
       [-2.0630000e+03,  1.3384000e+04, -1.5107286e+00, ...,
        -6.6015625e-01, -1.0003906e+01,  1.2187500e+00],
       [ 2.7570000e+03,  1.6008000e+04, -1.2309641e+00, ...,
        -6.6015625e-01, -1.0003906e+01,  1.2187500e+00],
       ...,
       [-2.5592000e+04, -7.3242000e+04, -1.0446408e+02, ...,
         3.6328125e-01, -1.4648438e+00, -9.0000000e+00],
       [ 4.0027000e+04, -1.7502000e+04, -9.4336610e+01, ...,
         3.6328125e-01, -1.4648438e+00, -9.0000000e+00],
       [ 7.9369000e+04,  7.7820000e+03, -9.4336610e+01, ...,
         3.6328125e-01, -1.4648438e+00, -9.0000000e+00]])

In [7]:
timesteps = 200
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback = timesteps)

n_features = 8
X = np.array(X)
X = X.reshape(X.shape[0], timesteps, n_features)

X

array([[[ 2.75700000e+03,  1.60080000e+04, -1.23096410e+00, ...,
         -6.60156250e-01, -1.00039060e+01,  1.21875000e+00],
        [ 7.50400000e+03,  5.64400000e+03, -1.23096410e+00, ...,
         -6.60156250e-01, -1.00039060e+01,  1.21875000e+00],
        [-3.85500000e+03, -1.58930000e+04, -1.06310520e+00, ...,
         -6.60156250e-01, -1.00039060e+01,  1.21875000e+00],
        ...,
        [ 1.94680000e+04,  4.02400000e+03,  1.88561300e+01, ...,
          1.24023440e+01, -1.36406250e+01,  2.90625000e+00],
        [ 3.62100000e+03,  5.57500000e+03,  1.48275210e+01, ...,
          1.24023440e+01, -1.36406250e+01,  2.90625000e+00],
        [-3.11310000e+04, -1.44100000e+03,  1.48275210e+01, ...,
          1.24023440e+01, -1.36406250e+01,  2.90625000e+00]],

       [[ 7.50400000e+03,  5.64400000e+03, -1.23096410e+00, ...,
         -6.60156250e-01, -1.00039060e+01,  1.21875000e+00],
        [-3.85500000e+03, -1.58930000e+04, -1.06310520e+00, ...,
         -6.60156250e-01, -1.00039060e

In [8]:
# define model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(32, activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 64)           18688     
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 200, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200, 32)           8320      
_________________________________________________________________
lstm_3 (LSTM)                (None, 200, 64)           24832     
_________________________________________________________________
time_distributed (TimeDistri (None, 200, 8)            520       
Total params: 64,776
Trainable params: 64,776
Non-trainable params: 0
____________________________________________________

In [9]:
# fit model
model.fit(X, X, epochs=100, batch_size=10, verbose=1)
# demonstrate reconstruction
yhat = model.predict(X, verbose=1)
# print('---Predicted---')
# print(np.round(yhat,3))
# print('---Actual---')
# print(np.round(X, 3))

Epoch 1/100
1478/1478 [==============================] - 155s 105ms/step - loss: nan
Epoch 2/100
1478/1478 [==============================] - 157s 106ms/step - loss: nan
Epoch 3/100
1478/1478 [==============================] - 153s 104ms/step - loss: nan
Epoch 4/100
1478/1478 [==============================] - 154s 104ms/step - loss: nan
Epoch 5/100
1478/1478 [==============================] - 154s 104ms/step - loss: nan
Epoch 6/100
1478/1478 [==============================] - 156s 106ms/step - loss: nan
Epoch 7/100
1478/1478 [==============================] - 157s 106ms/step - loss: nan
Epoch 8/100
1478/1478 [==============================] - 155s 105ms/step - loss: nan
Epoch 9/100
1478/1478 [==============================] - 150s 101ms/step - loss: nan
Epoch 10/100
1478/1478 [==============================] - 154s 104ms/step - loss: nan
Epoch 11/100
1478/1478 [==============================] - 159s 107ms/step - loss: nan
Epoch 12/100
1478/1478 [==============================] - 159s 

1478/1478 [==============================] - 148s 100ms/step - loss: nan
Epoch 97/100
1478/1478 [==============================] - 148s 100ms/step - loss: nan
Epoch 98/100
1478/1478 [==============================] - 148s 100ms/step - loss: nan
Epoch 99/100
1478/1478 [==============================] - 148s 100ms/step - loss: nan
Epoch 100/100
462/462 [==============================] - 16s 34ms/step


In [ ]:
#extract encoder

encoder = K.function([model.layers[0].input], [model.layers[2].output])

In [10]:
print(np.round(yhat, 3))

[[[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 ...

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]]

 [[nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  ...
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan nan]
  [nan nan nan ... nan nan n